# 🚀 Delta Sharing and Real-Time Inference with Unity Catalog Models

This notebook demonstrates the full flow of consuming a shared model via Delta Sharing and deploying it for real-time inference.

Steps covered:

1. **Accept the shared Delta Share**  
   - Programmatically create a catalog from the shared model using Delta Sharing APIs.

2. **Ad-hoc Inference from Model Registry**  
   - Load the shared model directly from the Unity Catalog Model Registry.
   - Perform batch inference using the model within the cluster, assuming cluster environment compatibility.

3. **Deploy a Real-Time Model Serving Endpoint**  
   - Programmatically create a model serving endpoint using Databricks REST APIs.
   - Host the shared model as a RESTful API for scalable, production-grade inference.

4. **Perform Inference via REST API**  
   - Send data to the deployed endpoint.
   - Receive predictions programmatically using authentication tokens.

This end-to-end workflow highlights how Delta Sharing and Unity Catalog enable cross-workspace model consumption, deployment, and real-time inferencing — all without relying on manual UI actions.


In [0]:
%sql
-- Step 0: Accept Delta Share (Run once using SQL)
CREATE CATALOG IF NOT EXISTS abooth_delta_share
USING SHARE `databricks-field-eng`.`abooth_bike_sharing_model_share`
COMMENT "Catalog for accessing shared bike sharing model.";


## 📦 Load and Use the Shared Model (Ad-hoc Inference)

Now that the Delta Share has been accepted and the shared catalog is available, we can load the registered model directly from Unity Catalog.

In this section:
- Configure MLflow to use Unity Catalog as the model registry.
- Load the shared model using the model URI and alias (`@staging`).
- Prepare a small sample of new data that matches the training schema.
- Perform ad-hoc predictions directly on the cluster without deploying a separate endpoint.

This demonstrates how quickly models shared via Delta Sharing can be used for immediate inference inside a recipient workspace.


In [0]:
# Step 1: Load Model from Unity Catalog (Recipient Workspace)
import mlflow
import pandas as pd

# Configure MLflow to use Databricks Unity Catalog
mlflow.set_registry_uri("databricks-uc")

# Load the model using the shared catalog
model_uri = "models:/abooth_delta_share.default.bike_sharing_uc_model@staging"
loaded_model = mlflow.sklearn.load_model(model_uri)

# Step 2: Prepare New Data
new_data = pd.DataFrame({
    'season': [1],
    'yr': [0],
    'mnth': [1],
    'hr': [10],
    'holiday': [0],
    'weekday': [3],
    'workingday': [1],
    'weathersit': [1],
    'temp': [0.3],
    'atemp': [0.31],
    'hum': [0.5],
    'windspeed': [0.1]
})

# Step 3: Make Predictions
prediction = loaded_model.predict(new_data)
print(f"Predicted bike rentals: {prediction[0]}")


[LightGBM] [Warning] lambda_l2 is set=353.1987939539919, reg_lambda=0.0 will be ignored. Current value: lambda_l2=353.1987939539919
[LightGBM] [Warning] lambda_l1 is set=5.441545096322337, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5.441545096322337
Predicted bike rentals: 48.69329265699597


## 🚀 Deploy the Shared Model to Real-Time Serving

In this section, we deploy the shared and registered model as a real-time serving endpoint using Databricks Model Serving.

Steps:
- Programmatically create a new serving endpoint via the Databricks REST API.
- Configure the endpoint to serve the shared model version from Unity Catalog.
- Enable auto-scaling options like `scale_to_zero` for cost efficiency.

Once the endpoint is active, we will send inference requests over HTTP by authenticating with a Databricks personal access token (PAT).  
This enables scalable, production-grade, low-latency predictions from the shared model.



In [0]:
import requests
import json

# Step 1: Define variables
DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
WORKSPACE_URL = "https://xxxxxxxx.azuredatabricks.net"  # your workspace
headers = {
    "Authorization": f"Bearer {DATABRICKS_TOKEN}",
    "Content-Type": "application/json",
}

# Step 2: Define your serving endpoint config
endpoint_name = "abooth_bike_serving_endpoint"

create_payload = {
    "name": endpoint_name,
    "config": {
        "served_models": [
            {
                "model_name": "abooth_delta_share.default.bike_sharing_uc_model",
                "model_version": "1",  # Use version number
                "workload_type": "CPU",  # or "GPU"
                "workload_size": "Small",  # Small / Medium / Large
                "scale_to_zero_enabled": True,
            }
        ]
    }
}

# Step 3: Make the request to create endpoint
create_url = f"{WORKSPACE_URL}/api/2.0/serving-endpoints"
response = requests.post(create_url, headers=headers, data=json.dumps(create_payload))

# Step 4: Check result
if response.status_code == 200:
    print(f"Successfully created serving endpoint: {endpoint_name}")
else:
    print(f"Failed to create serving endpoint: {response.text}")


Successfully created serving endpoint: abooth_bike_serving_endpoint


In [0]:
import requests
import json

# Step 1: Authenticate to Databricks by retrieving the notebook's API token
DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

# Step 2: Define the workspace URL and model serving endpoint URL
WORKSPACE_URL = "https://xxxxxxxx.azuredatabricks.net"  # your workspace
ENDPOINT = f"{WORKSPACE_URL}/serving-endpoints/abooth_bike_serving_endpoint/invocations"

# Step 3: Set the HTTP headers, including the authorization token
headers = {
    "Authorization": f"Bearer {DATABRICKS_TOKEN}",
    "Content-Type": "application/json",
}

# Step 4: Build the request payload by converting the input DataFrame to the required format
payload = {"dataframe_split": new_data.to_dict(orient="split")}

# Step 5: Send the POST request to the serving endpoint for inference
response = requests.post(ENDPOINT, headers=headers, data=json.dumps(payload))

# Step 6: Parse and display the prediction results
response.json()

{'error_code': 'RESOURCE_DOES_NOT_EXIST',
 'message': 'The given endpoint does not exist.'}

# 🏁 Conclusion and Next Steps

In this notebook, we demonstrated how to:

- Accept a Delta Share containing a Unity Catalog-registered model.
- Load and perform ad-hoc inference on the shared model.
- Deploy the shared model to a real-time serving endpoint.
- Perform predictions using a REST API with secure authentication.

---

## 📈 Next Steps: A/B Testing with Shared Models

If multiple models are available — for example, if a previously shared **production model** exists alongside a newly shared **candidate model** — we can extend this workflow to enable **A/B testing**.

Potential next steps include:

- **Create multiple serving endpoints** for the "Prod" model and the "Staging" model.
- **Route traffic** dynamically between models based on a defined split (e.g., 50/50, 80/20).
- **Log model predictions and outcomes** into a centralized Delta table for comparison.
- **Analyze A/B test results** to determine which model performs better on real-world data.
- **Promote the winning model** by updating aliases (e.g., setting the `@prod` alias) and refreshing the serving endpoint.

By using Delta Sharing and Unity Catalog together, model consumers can easily experiment with multiple models across workspaces without needing to duplicate or manually manage artifacts.

---
